In [1]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup as bs

os.chdir("/home/ubuntu/real_estate_dashboard")
print(os.getcwd())

df = pd.read_csv("./0_data/legal_info_b.csv")
df

/home/ubuntu/real_estate_dashboard


,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,법정동시군구코드,법정동읍면동코드,주소
0,1111010100,서울특별시,종로구,청운동,NaN,19880423,,11110,10100,서울특별시 종로구 청운동
1,1111010200,서울특별시,종로구,신교동,NaN,19880423,,11110,10200,서울특별시 종로구 신교동
2,1111010300,서울특별시,종로구,궁정동,NaN,19880423,,11110,10300,서울특별시 종로구 궁정동
3,1111010400,서울특별시,종로구,효자동,NaN,19880423,,11110,10400,서울특별시 종로구 효자동
4,1111010500,서울특별시,종로구,창성동,NaN,19880423,,11110,10500,서울특별시 종로구 창성동
...,...,...,...,...,...,...,...,...,...,...
20273,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,,50130,32022,제주특별자치도 서귀포시 표선면 하천리
20274,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,,50130,32023,제주특별자치도 서귀포시 표선면 성읍리
20275,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,,50130,32024,제주특별자치도 서귀포시 표선면 가시리
20276,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,,50130,32025,제주특별자치도 서귀포시 표선면 세화리


In [2]:
# 생성한 법정코드 고윳값 찾기
LAWD_CD_list = df['법정동시군구코드'].unique()
print(len(LAWD_CD_list))
LAWD_CD_list

250


array([11110, 11140, 11170, 11200, 11215, 11230, 11260, 11290, 11305,
       11320, 11350, 11380, 11410, 11440, 11470, 11500, 11530, 11545,
       11560, 11590, 11620, 11650, 11680, 11710, 11740, 26110, 26140,
       26170, 26200, 26230, 26260, 26290, 26320, 26350, 26380, 26410,
       26440, 26470, 26500, 26530, 26710, 27110, 27140, 27170, 27200,
       27230, 27260, 27290, 27710, 28110, 28140, 28177, 28185, 28200,
       28237, 28245, 28260, 28710, 28720, 29110, 29140, 29155, 29170,
       29200, 30110, 30140, 30170, 30200, 30230, 31110, 31140, 31170,
       31200, 31710, 36110, 41111, 41113, 41115, 41117, 41131, 41133,
       41135, 41150, 41171, 41173, 41190, 41210, 41220, 41250, 41271,
       41273, 41281, 41285, 41287, 41290, 41310, 41360, 41370, 41390,
       41410, 41430, 41450, 41461, 41463, 41465, 41480, 41500, 41550,
       41570, 41590, 41610, 41630, 41650, 41670, 41800, 41820, 41830,
       42110, 42130, 42150, 42170, 42190, 42210, 42230, 42720, 42730,
       42750, 42760,

In [4]:
import time

# 실행 전 시간 기록
start_time = time.time()

# API 키 세팅
api_key = 'odgWuZqEkLq3B8i3URl%2BvXJl42Lr61W8sPd0edRlzlvelJmFSPtJ%2B%2BpdQXpU8lhVdGN1yvYOHsuRJNH8G3m6aQ%3D%3D'

# 컬럼
columns = ['거래금액',
'거래유형',
'건축년도',
'년',
'단지',
'법정동',
'시군구',
'월',
'일',
'전용면적',
'중개사소재지',
'지번',
'지역코드',
'층']

# 서비스 url
url = "http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcOffiTrade"

# 에러 추출용
error = []

# 데이터 저장용
total = pd.DataFrame()

for i in range(len(LAWD_CD_list)):
    
    # 데이터를 담는 리스트
    df_total = []
    
    # 추출하는 연월
    year_month_key = "202101"

    # 요청변수 파라미터 설정
    params = "?" + \
             "serviceKey=" + api_key + "&" + \
             "LAWD_CD=" + str(LAWD_CD_list[i]) + "&" + \
             "DEAL_YMD=" + year_month_key
    
    res = requests.get(url + params)
    soup = bs(res.text, 'xml')
    items = soup.find_all('item')
    
    if(len(items) > 0):
        # 수집된 데이터를 데이터 프레임 형식으로 변환
        for j in range(len(items)):
            df_raw = []
            for column in columns:
                # 컬럼 데이터가 없는 경우 처리
                try:
                    df_raw.append(items[j].find(column).text)
                except:
                    print(LAWD_CD_list[i], j , column) # 에러 컬럼 파악 여부
                    df_raw.append('없음')
            # 데이터 행 하나를 더해줍니다        
            df_total.append(df_raw)
            
    else:
        # 해당 지역에 거래 데이터가 없는 경우
        error.append(LAWD_CD_list[i])
        continue
        
    df = pd.DataFrame(df_total)
    df.columns = columns
    
    total = pd.concat([total,df])
    total.columns = columns
    
    print(f"{LAWD_CD_list[i]} 데이터 추출 완료")
        
        
        
        
df = pd.read_csv("./0_data/legal_info_b.csv")      

for i in range(len(error)):
     print(f'{error[i]} 거래 데이터가 없습니다')
    
# 실행 후 시간 기록
end_time = time.time()    
    
# 실행 시간 계산
execution_time = end_time - start_time
print(f"실행 시간: {execution_time} 초")   
    
total.reset_index(drop=True,inplace=True)
total

11110
11140
11170 16 건축년도
11170 20 건축년도
11170 27 건축년도
11170 28 건축년도
11170 31 건축년도
11170
11200
11215
11230 4 건축년도
11230 5 건축년도
11230 10 건축년도
11230 11 건축년도
11230 17 건축년도
11230 19 건축년도
11230 28 건축년도
11230 35 건축년도
11230
11260 4 건축년도
11260 5 건축년도
11260
11290
11305
11320 43 건축년도
11320
11350
11380 14 건축년도
11380
11410
11440 22 건축년도
11440 23 건축년도
11440 25 건축년도
11440
11470
11500
11530 22 건축년도
11530
11545
11560 166 건축년도
11560
11590
11620
11650 72 지번
11650 74 지번
11650
11680
11710 7 건축년도
11710 9 건축년도
11710 10 건축년도
11710 11 건축년도
11710 12 건축년도
11710 25 건축년도
11710 27 건축년도
11710 31 건축년도
11710 32 건축년도
11710 35 건축년도
11710 37 건축년도
11710 38 건축년도
11710 39 건축년도
11710 40 건축년도
11710 41 건축년도
11710 43 건축년도
11710 44 건축년도
11710 48 건축년도
11710 50 건축년도
11710 51 건축년도
11710 53 건축년도
11710 54 건축년도
11710 55 건축년도
11710 59 건축년도
11710 61 건축년도
11710 62 건축년도
11710 65 건축년도
11710 70 건축년도
11710 71 건축년도
11710 72 건축년도
11710 79 건축년도
11710 80 건축년도
11710 82 건축년도
11710 83 건축년도
11710 84 건축년도
11710 87 건축년도
11710 88 건축년도
11710 90 건축년도
117

,거래금액,거래유형,건축년도,년,단지,법정동,시군구,월,일,전용면적,중개사소재지,지번,지역코드,층
0,"45,000",,1994,2021,도렴구역 제18지구,내수동,종로구,1,5,67.36,,167,11110,8
1,"27,900",,2004,2021,경희궁의 아침 2단지,내수동,종로구,1,6,32.43,,71,11110,4
2,"26,500",,2004,2021,경희궁의 아침 2단지,내수동,종로구,1,9,32.43,,71,11110,5
3,"31,500",,2004,2021,용비어천가,내수동,종로구,1,22,33,,75,11110,17
4,"29,150",,2004,2021,광화문시대,내수동,종로구,1,23,34.9,,74,11110,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5241,"22,000",,2020,2021,예지퍼스트,서귀동,서귀포시,1,28,65.82,,433-4,50130,4
5242,"13,500",,2017,2021,더프라임,법환동,서귀포시,1,17,31.94,,747-1,50130,6
5243,"11,500",,2020,2021,리움 오피스텔,동홍동,서귀포시,1,18,18.76,,509-3,50130,4
5244,"6,000",,없음,2021,서강파인힐5차,서홍동,서귀포시,1,5,24.65,,285-3,50130,1


In [5]:
directory = '0_data/offi_trade'

if not os.path.exists(directory):
    os.makedirs(directory)

file_name = f"{directory}/offi_trade_{str(year_month_key)}.csv"

total.to_csv(file_name, index = False)